In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd

import efd_utils
import df_utils

In [ ]:
location = "summit"
#location = "tucson"

In [ ]:
client = efd_utils.get_client(location)

In [ ]:
csc = efd_utils.CSC("Environment", 1)
#csc = efd_utils.CSC.from_entry("ATArchiver")
#csc = efd_utils.CSC.from_entry("ATPtg")
#csc = efd_utils.CSC.from_entry("ATMCS")
#csc = efd_utils.CSC.from_entry("ATSpectrograph")
d1 = datetime(2019, 11, 26)
d2 = datetime(2019, 12, 10)
summary_state = 3  # FAULT
time_window = 10.0  # seconds
time_format = '%Y-%m-%dT%H:%M:%S.%f'

In [ ]:
query = efd_utils.get_base_query(columns=["private_sndStamp",
                                          "private_rcvStamp",
                                          "summaryState"],
                                 csc_name=csc.name,
                                 csc_index=csc.index,
                                 topic_name="logevent_summaryState")
# Handle indexed components from base query
if "WHERE" not in query:
    query += " WHERE"
if csc.index:
    query += " AND"
query += f" summaryState={summary_state}"
query += " AND " + efd_utils.get_time_clause(date_range=(d1, d2))
#print(query)
ss_df = await client.query(query)
try:
    ss_df = df_utils.convert_timestamps(ss_df, ["private_sndStamp",
                                                "private_rcvStamp"])
except KeyError:
    print("NO DATA FOUND!!!!")
#print(ss_df)

In [ ]:
query = efd_utils.get_base_query(columns=["private_sndStamp",
                                          "private_rcvStamp",
                                          "errorCode",
                                          "errorReport",
                                          "traceback"],
                                 csc_name=csc.name,
                                 csc_index=csc.index,
                                 topic_name="logevent_errorCode")
if "WHERE" not in query:
    query += " WHERE"
if csc.index:
    query += " AND"
query += " " + efd_utils.get_time_clause(date_range=(d1, d2))
#print(query)
ec_df = await client.query(query)
ec_df = df_utils.convert_timestamps(ec_df, ["private_sndStamp",
                                            "private_rcvStamp"])
#print(ec_df)

In [ ]:
timedelta = pd.Timedelta(time_window, unit='sec')

In [ ]:
if ss_df.private_sndStamp_date.size:
    for timestamp in ss_df.private_sndStamp_date:
        print("######################################")
        print(f"Fault Summary State: {timestamp.strftime(time_format)}")
        t1 = timestamp - timedelta
        t2 = timestamp + timedelta
        error_indexes = ec_df.private_sndStamp_date.to_numpy()
        index = np.logical_and(error_indexes > t1, error_indexes < t2)
        error = ec_df[index]
        if error.shape[0] > 1:
            print("Multiple errorCode messages associated with Fault State!")
        for entry in range(error.shape[0]):
            print("----------------------------------")
            snd_stamp = error.private_sndStamp_date[entry].strftime(time_format)
            rcv_stamp = error.private_rcvStamp_date[entry].strftime(time_format)
            print(f"Fault State Report: {snd_stamp} ({rcv_stamp})")
            print(f"Error Code: {error.errorCode.values[entry]}")
            print(f"Error Report: {error.errorReport.values[entry]}")
            print(f"Traceback: {error.traceback.values[entry]}")
else:
    print(f"No fault states found for {csc}")